### Conversion of the Dataset

In [9]:
%pip install datasets huggingface_hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%huggingface-cli` not found.


In [2]:
from datasets import load_dataset

dataset = load_dataset('vmal/jobs_dataset')
print(dataset)

C:\Users\jaius\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text'],
        num_rows: 5579
    })
})


In [4]:

SYSTEM_PROMPT = "You are a helpful assistant that extracts structured job information from unstructured job postings."

def format_for_llama2(example):
    return {
        "text": f"<s>[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n{example['source_text']}\n[/INST]\n{example['target_text']}</s>"
    }

formatted_dataset = dataset.map(format_for_llama2)

# If you only need the 'text' column for training:
formatted_dataset = formatted_dataset.remove_columns([col for col in formatted_dataset["train"].column_names if col != "text"])

Map: 100%|████████████████████████████████████████████████████████████████| 5579/5579 [00:00<00:00, 9719.45 examples/s]


In [5]:
print(formatted_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5579
    })
})


In [6]:
formatted_dataset = formatted_dataset['train'].train_test_split(test_size = 0.1)

In [7]:
print(formatted_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5021
    })
    test: Dataset({
        features: ['text'],
        num_rows: 558
    })
})


**Push Dataset to HF**

In [14]:
from huggingface_hub import login
import os

login(os.getenv('HF_TOKEN'))

In [17]:
# Remove columns starting with "_"
cols_to_remove = [c for c in formatted_dataset.column_names if c.startswith("_")]
formatted_dataset = formatted_dataset.remove_columns(cols_to_remove)

# Now push to Hub
formatted_dataset.push_to_hub(
    "01fragtos/job_text_to_json_llama-2",
    private=False
)

Uploading the dataset shards: 100%|█████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.35s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/01fragtos/job_text_to_json_llama-2/commit/b6d43c6a15265e6b2848f4629f7c00b2bd8250f3', commit_message='Upload dataset', commit_description='', oid='b6d43c6a15265e6b2848f4629f7c00b2bd8250f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/01fragtos/job_text_to_json_llama-2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='01fragtos/job_text_to_json_llama-2'), pr_revision=None, pr_num=None)